# Introduction

This is my first NLP project, the goal is to named entity in a sentence.

In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
data = pd.read_csv("ner_dataset.csv", encoding="unicode-escape")
data = data.fillna(method="ffill")

print("Number of sentences: ", len(data.groupby(['Sentence #'])))

words = list(set(data["Word"].values))
n_words = len(words)
print("Number of words in the dataset: ", n_words)

tags = list(set(data["Tag"].values))
print("Tags:", tags)
n_tags = len(tags)
print("Number of Labels: ", n_tags)

print("What the dataset looks like:")
# Show the first 10 rows
data.head(n=10) 

C:\Users\anton\AppData\Local\Temp\ipykernel_31104\2338901192.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="ffill")


Number of sentences:  47959
Number of words in the dataset:  35177
Tags: ['B-tim', 'O', 'I-org', 'I-geo', 'I-art', 'I-per', 'B-gpe', 'B-nat', 'I-eve', 'B-per', 'B-geo', 'B-art', 'I-tim', 'B-org', 'I-nat', 'B-eve', 'I-gpe']
Number of Labels:  17
What the dataset looks like:


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [13]:
# Exploration des données : afficher quelques exemples de phrases et leurs étiquettes associées
sample_sentences = data.groupby('Sentence #').apply(lambda sentence: ' '.join(sentence['Word']))
sample_labels = data.groupby('Sentence #').apply(lambda sentence: ' '.join(sentence['Tag']))
for i in range(3):
    print("Exemple de phrase :", sample_sentences[i])
    print("Étiquettes associées :", sample_labels[i])
    print()

vocab = set(data['Word'].unique())

tag_to_idx = {tag: idx for idx, tag in enumerate(data['Tag'].unique())}

# Transformation des données en format modèle : conversion des mots en indices et encodage des étiquettes de tags
data['Word_idx'] = data['Word'].map({word: idx for idx, word in enumerate(vocab)})
data['Tag_idx'] = data['Tag'].map(tag_to_idx)

C:\Users\anton\AppData\Local\Temp\ipykernel_31104\3544627477.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_sentences = data.groupby('Sentence #').apply(lambda sentence: ' '.join(sentence['Word']))
C:\Users\anton\AppData\Local\Temp\ipykernel_31104\3544627477.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_labels = data.groupby('Sentence #').apply(lambda sentence: ' '.join(sentence['Ta

Exemple de phrase : Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
Étiquettes associées : O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O

Exemple de phrase : Iranian officials say they expect to get access to sealed sensitive parts of the plant Wednesday , after an IAEA surveillance system begins functioning .
Étiquettes associées : B-gpe O O O O O O O O O O O O O O B-tim O O O B-org O O O O O

Exemple de phrase : Helicopter gunships Saturday pounded militant hideouts in the Orakzai tribal region , where many Taliban militants are believed to have fled to avoid an earlier military offensive in nearby South Waziristan .
Étiquettes associées : O O B-tim O O O O O B-geo O O O O O B-org O O O O O O O O O O O O O O B-geo I-geo O

    Sentence #           Word  POS Tag  Word_idx  Tag_idx
0  Sentence: 1      Thousands  NNS   O     22205        0
1  Sentence: 1             of   IN   

In [15]:
# Transformation des données en format modèle : conversion des mots en indices et encodage des étiquettes de tags
pad_tokens = data['Word_idx']
pad_tags = data['Tag_idx']

In [16]:
from sklearn.model_selection import train_test_split

tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)

In [17]:
input_dim = len(vocab)

In [18]:
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

In [22]:
model = Sequential()

# Add Embedding layer
model.add(Embedding(input_dim=input_dim, output_dim=64, input_length=75))

# Add bidirectional LSTM
model.add(Bidirectional(LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))

# Add LSTM
model.add(LSTM(units=64, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

# Add timeDistributed Layer
model.add(TimeDistributed(Dense(n_tags, activation="relu")))

#Optimiser 
# adam = k.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

c:\Users\anton\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ ?                      │   0 (unbuilt) │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [4]:
from itertools import chain

def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))
    
    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok
token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

In [8]:
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)
data_fillna = data.fillna(method='ffill', axis=0)

C:\Users\anton\AppData\Local\Temp\ipykernel_35556\3363008921.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_fillna = data.fillna(method='ffill', axis=0)


In [11]:
data_group = data_fillna.groupby(
    ['Sentence #'], as_index=False
)[['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx']].agg(lambda x: list(x))


In [14]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

def get_pad_train_test_val(data_group, data):

    #get max token and tag length
    n_token = len(list(set(data['Word'].to_list())))
    n_tag = len(list(set(data['Tag'].to_list())))

    #Pad tokens (X var)    
    tokens = data_group['Word_idx'].tolist()
    maxlen = max([len(s) for s in tokens])
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= n_token - 1)

    #Pad Tags (y var) and convert it into one hot encoding
    tags = data_group['Tag_idx'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag2idx["O"])
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]
    
    #Split train, test and validation set
    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tokens length:', len(train_tokens),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags),
    )
    
    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data_group, data)

train_tokens length: 32372 
train_tokens length: 32372 
test_tokens length: 4796 
test_tags: 4796 
val_tokens: 10791 
val_tags: 10791


In [16]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
from numpy.random import seed
seed(1)
tensorflow.random.set_seed(2)

In [18]:
input_dim = len(list(set(data['Word'].to_list())))+1
print(input_dim)
output_dim = 64
input_length = max([len(s) for s in data_group['Word_idx'].tolist()])
print(input_length)
n_tags = len(tag2idx)
print(n_tags)

35178
104
17


In [19]:
model = Sequential()

# Add Embedding layer
model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))

# Add bidirectional LSTM
model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))

# Add LSTM
model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

# Add timeDistributed Layer
model.add(TimeDistributed(Dense(n_tags, activation="relu")))

#Optimiser 
# adam = k.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 104, 64)           2251392   
                                                                 
 bidirectional (Bidirectiona  (None, 104, 128)         66048     
 l)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 104, 64)           49408     
                                                                 
 time_distributed (TimeDistr  (None, 104, 17)          1105      
 ibuted)                                                         
                                                                 
Total params: 2,367,953
Trainable params: 2,367,953
Non-trainable params: 0
_________________________________________________________________


In [22]:
hist = model.fit(train_tokens, np.array(train_tags), batch_size=1000, verbose=1, epochs=10, validation_split=0.2)

Epoch 1/10
26/26 [==============================] - 222s 8s/step - loss: 1.0109 - accuracy: 0.9247 - val_loss: 0.3248 - val_accuracy: 0.9681
Epoch 2/10
26/26 [==============================] - 294s 11s/step - loss: 0.3233 - accuracy: 0.9677 - val_loss: 0.2740 - val_accuracy: 0.9681
Epoch 3/10
26/26 [==============================] - 352s 14s/step - loss: 0.2685 - accuracy: 0.9678 - val_loss: 0.2350 - val_accuracy: 0.9681
Epoch 4/10
26/26 [==============================] - 400s 15s/step - loss: 0.2526 - accuracy: 0.9678 - val_loss: 0.2378 - val_accuracy: 0.9682
Epoch 5/10
26/26 [==============================] - 447s 17s/step - loss: 0.2350 - accuracy: 0.9678 - val_loss: 0.2187 - val_accuracy: 0.9682
Epoch 6/10
26/26 [==============================] - 8866s 354s/step - loss: 0.2204 - accuracy: 0.9678 - val_loss: 0.2088 - val_accuracy: 0.9681
Epoch 7/10
26/26 [==============================] - 37929s 1516s/step - loss: 0.4203 - accuracy: 0.9212 - val_loss: 0.6171 - val_accuracy: 0.8625
E

In [23]:
pred_cat = model.predict(test_tokens)
pred = np.argmax(pred_cat, axis=-1)
y_te_true = np.argmax(test_tags, -1)

150/150 [==============================] - 14s 88ms/step


In [33]:
from sklearn.metrics import classification_report

# Ajuster la forme de y_te_true et pred en unidimensionnel
y_te_true_flat = y_te_true.flatten()
pred_flat = pred.flatten()

# Obtenir le rapport de classification
report = classification_report(y_te_true_flat, pred_flat)

# Afficher le rapport de classification
print(report)


c:\Users\anton\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1628
           1       0.71      0.00      0.01      1715
           2       0.00      0.00      0.00        34
           3       0.00      0.00      0.00        23
           4       0.55      0.02      0.04      3690
           5       0.00      0.00      0.00        28
           6       0.00      0.00      0.00         7
           7       0.00      0.00      0.00      2038
           8       0.00      0.00      0.00        17
           9       0.00      0.00      0.00      1697
          10       0.00      0.00      0.00       674
          11       0.00      0.00      0.00      1994
          12       0.00      0.00      0.00        22
          13       0.00      0.00      0.00       681
          14       0.49      0.15      0.23      1697
          15       0.00      0.00      0.00        28
          16       0.97      1.00      0.98    482811

    accuracy              

c:\Users\anton\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\anton\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [63]:
import pickle

# Saving Vocab
with open('tokens_to_index.pickle', 'wb') as handle:
    pickle.dump(token2idx, handle, protocol=pickle.HIGHEST_PROTOCOL)
 
# Saving Vocab
with open('tag_to_index.pickle', 'wb') as handle:
    pickle.dump(tag2idx, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# Saving Model Weight
model.save_weights('entity_recognition.h5')

In [10]:
import spacy
# Load the trained spaCy NER model from the specified path
nlp = spacy.load('entity_recognition.h5')

OSError: [E053] Could not read meta.json from entity_recognition.h5